In [1]:
import pandas as pd
import pandas_profiling

%matplotlib inline

In [2]:
train = pd.read_csv("../data/train_test/train.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1911 entries, 0 to 1910
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   category  1911 non-null   object 
 1   type      1911 non-null   object 
 2   year      1911 non-null   int64  
 3   month     1911 non-null   object 
 4   value     1911 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 74.8+ KB


In [4]:
train.year.value_counts()

2020    91
2009    91
2001    91
2002    91
2003    91
2004    91
2005    91
2006    91
2007    91
2008    91
2010    91
2019    91
2011    91
2012    91
2013    91
2014    91
2015    91
2016    91
2017    91
2018    91
2000    91
Name: year, dtype: int64

# EDA

To avoid *overfitting* with our insights, we use only a half of the training set for exploration.

In [5]:
eda_df = train[train.year <= 2010]

In [6]:
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 130 to 1910
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   category  1001 non-null   object 
 1   type      1001 non-null   object 
 2   year      1001 non-null   int64  
 3   month     1001 non-null   object 
 4   value     1001 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 46.9+ KB


In [7]:
eda_df['category'].value_counts()

Verkehrsunfälle    429
Alkoholunfälle     286
Fluchtunfälle      286
Name: category, dtype: int64

In [8]:
eda_df['type'].value_counts()

insgesamt                 429
Verletzte und Getötete    429
mit Personenschäden       143
Name: type, dtype: int64

In [9]:
eda_df['year'].value_counts()

2010    91
2009    91
2008    91
2007    91
2006    91
2005    91
2004    91
2003    91
2002    91
2001    91
2000    91
Name: year, dtype: int64

In [10]:
eda_df['month'].value_counts()

Summe     77
200412     7
200202     7
200201     7
200312     7
          ..
200703     7
200702     7
200701     7
200812     7
200012     7
Name: month, Length: 133, dtype: int64

In [11]:
# month == 'Summe' contains values aggregated yearly
# these rows need to be dropped before training
eda_df[eda_df['month'] == 'Summe'].groupby('year').sum() == eda_df[eda_df['month'] != 'Summe'].groupby('year').sum()

,value
year,
2000,True
2001,True
2002,True
2003,True
2004,True
2005,True
2006,True
2007,True
2008,True


In [12]:
# month values are of incorrect format
# use regex to extract month number
eda_df['month'].str.extract(r'\d\d\d\d(\d\d)')

,0
130,NaN
131,01
132,02
133,03
134,04
...,...
1906,08
1907,09
1908,10
1909,11


In [13]:
eda_df['value'].describe()

count     1001.000000
mean      1375.538462
std       4394.788956
min          5.000000
25%         46.000000
50%        396.000000
75%        837.000000
max      45713.000000
Name: value, dtype: float64

In [19]:
subset_eda_df = eda_df[(eda_df['month'] != 'Summe')\
                       & (eda_df['category'] == 'Alkoholunfälle')\
                       & (eda_df['type'] == 'insgesamt')
                      ]
subset_eda_df.describe()

,year,value
count,132.000000,132.000000
mean,2005.000000,53.386364
std,3.174324,18.303067
min,2000.000000,14.000000
25%,2002.000000,40.000000
50%,2005.000000,51.000000
75%,2008.000000,65.000000
max,2010.000000,107.000000
